# Lesson 1 Practice - Cars

In this notebook I'll use lesson 1 knowledge to build an image classifier. The ultimate goal will be to identify the car model in the image from several car manufacturers - I'll start with the our german folks: VW, Audi, Mercedes-Benz, smart and BMW.  

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import fast.ai library:

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate

## Creating the Dataset